### Analysis on Lunar Date Conversions
* At this moment, there are 2 algorithms available
  * Algo 1 is based on Hong Kong Observatory's data
  * Algo 2 is based on astro-calculation, using VSOP87D and ELP2000-82B
* The 2 algorithms procude different results on year 1914, 1915, 1916, 1920

In [1]:
from common import LunarAlgo, get_lunar_year_info, get_supported_lunar_year_range

In [2]:
get_lunar_year_info(LunarAlgo.ALGO_1, 1914)

LunarYearInfo(first_day=datetime.date(1914, 1, 26), leap_month=5, month_lengths=[30, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30, 29, 30])

In [3]:
get_lunar_year_info(LunarAlgo.ALGO_2, 1914)

LunarYearInfo(first_day=datetime.date(1914, 1, 26), leap_month=5, month_lengths=[30, 30, 29, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30])

In [4]:
from functools import partial

# Use algo1's supported range since it's narrower.
# The year supported by algo1 should also be supported by algo2.
supported_range = get_supported_lunar_year_range(LunarAlgo.ALGO_1)
years = list(range(supported_range.start, supported_range.end + 1))

f1 = partial(get_lunar_year_info, LunarAlgo.ALGO_1)
f2 = partial(get_lunar_year_info, LunarAlgo.ALGO_2)

results_algo1 = list(map(f1, years))
results_algo2 = list(map(f2, years))

In [5]:
from itertools import compress

results_cmp = [r1 == r2 for r1, r2 in zip(results_algo1, results_algo2)]

unmatched = list(compress(
  years, [not x for x in results_cmp]
))
unmatched

[1914, 1915, 1916, 1920]

In [6]:
for y in unmatched:
  algo1_info = get_lunar_year_info(LunarAlgo.ALGO_1, y)
  algo2_info = get_lunar_year_info(LunarAlgo.ALGO_2, y)

  print(f'Examine for year {y}')
  if algo1_info.first_day != algo2_info.first_day:
    print(f'  algo1: {algo1_info.first_day}')
    print(f'  algo2: {algo2_info.first_day}')
    print()

  if algo1_info.leap_month != algo2_info.leap_month:
    print(f'  algo1: {algo1_info.leap_month}')
    print(f'  algo2: {algo2_info.leap_month}')
    print()

  if algo1_info.month_lengths != algo2_info.month_lengths:
    print(f'  algo1: {algo1_info.month_lengths}')
    print(f'  algo2: {algo2_info.month_lengths}')
    print()

Examine for year 1914
  algo1: [30, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30, 29, 30]
  algo2: [30, 30, 29, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30]

Examine for year 1915
  algo1: [30, 29, 30, 30, 29, 30, 29, 30, 29, 30, 29, 29]
  algo2: [30, 29, 30, 30, 29, 30, 29, 30, 29, 30, 29, 30]

Examine for year 1916
  algo1: 1916-02-03
  algo2: 1916-02-04

  algo1: [30, 30, 29, 30, 29, 30, 30, 29, 30, 29, 30, 29]
  algo2: [29, 30, 29, 30, 29, 30, 30, 29, 30, 29, 30, 29]

Examine for year 1920
  algo1: [29, 30, 29, 29, 30, 29, 29, 30, 29, 30, 30, 30]
  algo2: [29, 30, 29, 29, 30, 29, 29, 30, 30, 29, 30, 30]



* According to https://ytliu0.github.io/ChineseCalendar/computation_simp.html, these calculation results are not accurate
  * 10th lunar month in 1914
  * 1st lunar month in 1916
  * 10th lunar month in 1920
* That's why algo1 and algo2 have different results on these years

In [7]:
import common
from datetime import timedelta

new_moon_moments = common.new_moons_in_year(1914).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1914.11.17
# algo2 (calculated): 1914.11.18
new_moon_moments_ut1_8[-2]

datetime.datetime(1914, 11, 18, 0, 1, 49, 741625)

In [8]:
new_moon_moments = common.new_moons_in_year(1916).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1916.02.03
# algo2 (calculated): 1916.02.04
new_moon_moments_ut1_8[1]

datetime.datetime(1916, 2, 4, 0, 5, 13, 706552)

In [9]:
new_moon_moments = common.new_moons_in_year(1920).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1920.11.10
# algo2 (calculated): 1920.11.11
new_moon_moments_ut1_8[-2]

datetime.datetime(1920, 11, 11, 0, 4, 48, 680642)

In [10]:
from common import Jieqi, jieqi_moment
from dataclasses import dataclass

# As per https://ytliu0.github.io/ChineseCalendar/computation_simp.html,
# The following Jieqi moments are near 00:00:00, and are different than the ones that the government provides.
# The differences of Jieqi moments also makes differences in Ganzhi calendar, since Ganzhi calendar is based on Jieqi.

@dataclass
class JieqiPair:
  year: int
  jq: Jieqi

pairs = [
  JieqiPair(1912, Jieqi.小雪),
  JieqiPair(1913, Jieqi.秋分),
  JieqiPair(1917, Jieqi.大雪),
  JieqiPair(1927, Jieqi.白露),
  JieqiPair(1928, Jieqi.夏至),
  JieqiPair(1979, Jieqi.大寒),
]

results = [jieqi_moment(p.year, p.jq) for p in pairs]

moments = [r.moment for r in results]

moments_ut1_8 = [m + timedelta(hours=8) for m in moments]

moments_ut1_8

[datetime.datetime(1912, 11, 22, 23, 48, 14, 236353),
 datetime.datetime(1913, 9, 23, 23, 52, 48, 550398),
 datetime.datetime(1917, 12, 8, 0, 1, 5, 384493),
 datetime.datetime(1927, 9, 9, 0, 5, 30, 851009),
 datetime.datetime(1928, 6, 22, 0, 6, 27, 473321),
 datetime.datetime(1979, 1, 20, 23, 59, 56, 815078)]